In [ ]:
from dotenv import load_dotenv
import os

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
llm.invoke("How will the weather be in munich today?")

In [ ]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

In [ ]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? I would like to eat outside if possible"
)
result

In [ ]:
result.tool_calls

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [ ]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)

### Real third party API Call

In [ ]:
from langchain_core.tools import tool
import httpx



@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:5566/weather/{city}")

    if response.status_code == 200:
        return response.json().get("weather", "Weather information not available")
    else:
        return "Failed to get weather information"


@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:5566/outdoor-seating/{city}")

    if response.status_code == 200:
        return response.json().get(
            "outdoor_seating", "Outdoor seating information not available"
        )
    else:
        return "Failed to get outdoor seating information"


api_tools = [fake_weather_api, outdoor_seating_availability]
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [ ]:
from langchain_openai import ChatOpenAI


def interact_with_llm_and_tools(human_message: str):

    llm = ChatOpenAI()
    llm_with_tools_new = llm.bind_tools(api_tools)

    messages = [human_message]

    llm_output = llm_with_tools_new.invoke(messages)
    messages.append(llm_output)

    print("TOOL_CALLS:", llm_output.tool_calls)

    for tool_call in llm_output.tool_calls:
        tool_name = tool_call["name"].lower()
        tool = tool_mapping.get(tool_name)


        if tool:
            tool_output = tool.invoke(tool_call["args"])
            print("TOOL OUTPUT:", tool_output)
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm_with_tools_new.invoke(messages)
    return final_response

In [ ]:
interact_with_llm_and_tools(
    "How will the weather be in sunnyville today? I would like to eat outside if possible"
)

In [ ]:
interact_with_llm_and_tools(
    "How will the weather be in rainytown today? I would like to eat outside if possible"
)

In [ ]:
interact_with_llm_and_tools(
    "How will the weather be in munich today? I would like to eat outside if possible"
)

In [ ]:
interact_with_llm_and_tools("What´s the name of the Dad of the simpsons family?")